# Задание 3. Улучшение бейзлайнов и ItemItem
Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей  𝐾 .
Попробуйте стратегии ансамблирования изученных алгоритмов

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import log
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    #print(flags)
    
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [3]:
data = pd.read_csv('C:/Users/Yulia/Documents/Курсы/RecommendationSystem/Lesson2/data/retail_train.csv')


In [4]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


In [5]:
np.sort(popularity.sales_value)[::-1]

array([467993.62000001,  42645.75      ,  37981.91      , ...,
            0.        ,      0.        ,      0.        ])

In [6]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [7]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result['actual'] = result['actual'].apply(lambda x: list(x))
result.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


In [8]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [9]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
# Заведем фиктивный item_id (если юзер НЕ покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

#data_train.loc[data_train['item_id'].isin([999999]), 'quantity'] = 0
#data_train.head(10)

C:\Users\Yulia\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


# 1. Бейзлайны

In [10]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [11]:
%%time

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(top_5000, n=5))# фильтр по топ 5000
result.head(2)

Wall time: 801 ms


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1013407, 990941, 861445, 1129671, 877226]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[932562, 1052920, 957772, 935317, 1122112]"


In [12]:
result.head()

,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1013407, 990941, 861445, 1129671, 877226]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[932562, 1052920, 957772, 935317, 1122112]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[975155, 7409880, 971195, 13115161, 862826]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1082212, 845078, 1064919, 1126895, 958700]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[13008321, 851508, 12810422, 10150078, 1006184]"


In [13]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()


In [14]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train[data_train['item_id'].isin(top_5000)], n=5)# фильтр по топ 5000

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 364 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1013407, 990941, 861445, 1129671, 877226]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[932562, 1052920, 957772, 935317, 1122112]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [15]:
def weighted_random_recommendation(items_weights, n=5):
    item_id = np.array(items_weights.item_id)
   
    sum_=items_weights['weight'].sum()
    
    items_weights['pp']  = items_weights.apply(lambda row:row['weight']/sum_,axis = 1)
     
    recs = np.random.choice(item_id, size=n, replace=False,p = items_weights['pp'] )
        
    return recs.tolist()

In [16]:
items = data_train[data_train['item_id'].isin(top_5000)] # фильтр по топ 5000
items_weights = items.groupby('item_id')['sales_value'].sum().reset_index()

items_weights['weight'] = items_weights.apply(lambda row: log(row['sales_value'] if row['sales_value'] !=0 else 1) if log(row['sales_value'] if row['sales_value'] !=0 else 1)>=0 else 0,axis = 1)


In [17]:
%%time
weighted_r = weighted_random_recommendation(items_weights)
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_r)
result.head(2)

Wall time: 63.7 ms


,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1013407, 990941, 861445, 1129671, 877226]","[6534178, 6533889, 1029743, 6534166, 1082185]","[918733, 828139, 912233, 1017841, 982393]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[932562, 1052920, 957772, 935317, 1122112]","[6534178, 6533889, 1029743, 6534166, 1082185]","[918733, 828139, 912233, 1017841, 982393]"


# 2. Детерминированные алгоритмы item-item
## 2.1 Item-Item Recommender / ItemKNN

In [18]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='sum', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
~np.array([True, False])

array([False,  True])

In [20]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.3263136888428635

In [21]:
np.sort(data.item_id.unique())

array([   25671,    26081,    26093, ..., 18000012, 18024155, 18024556],
      dtype=int64)

In [22]:
# перенумеруем пользователей и товары
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [69]:
%%time

model = ItemItemRecommender(K=40, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


Wall time: 938 ms


In [70]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 128 ms


In [71]:
result.head(10)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1013407, 990941, 861445, 1129671, 877226]","[6534178, 6533889, 1029743, 6534166, 1082185]","[918733, 828139, 912233, 1017841, 982393]","[999999, 1082185, 981760, 995242, 1127831]","[1082185, 999999, 981760, 1127831, 961554]","[1082185, 981760, 1127831, 961554, 840361]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[932562, 1052920, 957772, 935317, 1122112]","[6534178, 6533889, 1029743, 6534166, 1082185]","[918733, 828139, 912233, 1017841, 982393]","[999999, 1082185, 981760, 1098066, 826249]","[1082185, 1098066, 826249, 981760, 999999]","[1082185, 1098066, 826249, 981760, 1044078]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[975155, 7409880, 971195, 13115161, 862826]","[6534178, 6533889, 1029743, 6534166, 1082185]","[918733, 828139, 912233, 1017841, 982393]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 961554]","[1082185, 981760, 999999, 1127831, 961554]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1082212, 845078, 1064919, 1126895, 958700]","[6534178, 6533889, 1029743, 6534166, 1082185]","[918733, 828139, 912233, 1017841, 982393]","[999999, 1082185, 981760, 995242, 1098066]","[1082185, 999999, 981760, 826249, 1098066]","[1082185, 981760, 826249, 999999, 840361]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[13008321, 851508, 12810422, 10150078, 1006184]","[6534178, 6533889, 1029743, 6534166, 1082185]","[918733, 828139, 912233, 1017841, 982393]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 999999, 981760, 1098066, 826249]","[1082185, 981760, 840361, 1098066, 999999]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[5583944, 924014, 1038663, 1008892, 840102]","[6534178, 6533889, 1029743, 6534166, 1082185]","[918733, 828139, 912233, 1017841, 982393]","[999999, 1082185, 981760, 995242, 1098066]","[1082185, 999999, 981760, 1098066, 826249]","[1082185, 981760, 1098066, 826249, 999999]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[865501, 944599, 8160430, 5585735, 951176]","[6534178, 6533889, 1029743, 6534166, 1082185]","[918733, 828139, 912233, 1017841, 982393]","[999999, 1082185, 981760, 995242, 1098066]","[1082185, 999999, 981760, 826249, 1098066]","[1082185, 981760, 840361, 826249, 1098066]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[913144, 1046816, 864615, 5572738, 1035136]","[6534178, 6533889, 1029743, 6534166, 1082185]","[918733, 828139, 912233, 1017841, 982393]","[999999, 1082185, 981760, 1098066, 826249]","[981760, 1082185, 999999, 1098066, 826249]","[1082185, 981760, 1098066, 826249, 1044078]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[1133556, 1086431, 836793, 6463464, 965050]","[6534178, 6533889, 1029743, 6534166, 1082185]","[918733, 828139, 912233, 1017841, 982393]","[999999, 1082185, 981760, 995242, 1127831]","[1082185, 999999, 981760, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]"
9,16,"[1062973, 1082185, 13007710]","[915349, 9802914, 1001646, 1022097, 1112754]","[6534178, 6533889, 1029743, 6534166, 1082185]","[918733, 828139, 912233, 1017841, 982393]","[999999, 1082185, 981760, 995242, 840361]","[1082185, 999999, 981760, 1098066, 826249]","[1082185, 999999, 981760, 1098066, 826249]"


In [72]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1013407, 990941, 861445, 1129671, 877226]","[6534178, 6533889, 1029743, 6534166, 1082185]","[918733, 828139, 912233, 1017841, 982393]","[999999, 1082185, 981760, 995242, 1127831]","[1082185, 999999, 981760, 1127831, 961554]","[1082185, 981760, 1127831, 961554, 840361]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[932562, 1052920, 957772, 935317, 1122112]","[6534178, 6533889, 1029743, 6534166, 1082185]","[918733, 828139, 912233, 1017841, 982393]","[999999, 1082185, 981760, 1098066, 826249]","[1082185, 1098066, 826249, 981760, 999999]","[1082185, 1098066, 826249, 981760, 1044078]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[975155, 7409880, 971195, 13115161, 862826]","[6534178, 6533889, 1029743, 6534166, 1082185]","[918733, 828139, 912233, 1017841, 982393]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 961554]","[1082185, 981760, 999999, 1127831, 961554]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1082212, 845078, 1064919, 1126895, 958700]","[6534178, 6533889, 1029743, 6534166, 1082185]","[918733, 828139, 912233, 1017841, 982393]","[999999, 1082185, 981760, 995242, 1098066]","[1082185, 999999, 981760, 826249, 1098066]","[1082185, 981760, 826249, 999999, 840361]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[13008321, 851508, 12810422, 10150078, 1006184]","[6534178, 6533889, 1029743, 6534166, 1082185]","[918733, 828139, 912233, 1017841, 982393]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 999999, 981760, 1098066, 826249]","[1082185, 981760, 840361, 1098066, 999999]"


In [73]:
#косинусово сходство

In [74]:
%%time

model = CosineRecommender(K=40, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


Wall time: 981 ms


In [75]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 172 ms


In [76]:
#TF-IDF взвешивание и TFIDFRecommender

In [77]:
%%time

model = TFIDFRecommender(K=40, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


Wall time: 961 ms


In [78]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

Wall time: 198 ms


In [79]:
result.to_csv('preds.csv', index=False)

In [34]:
result.apply(lambda x: precision_at_k(x['popular_recommendation'], x['actual'],  5), axis=1).mean()

0.15523996082272082

In [35]:
result.apply(lambda x: precision_at_k(x['random_recommendation'], x['actual'],  5), axis=1).mean()

0.006856023506366299

In [ ]:
result.apply(lambda x: precision_at_k(x['weighted_random_recommendation'], x['actual'],  5), axis=1).mean()

при К=20
itemitem = 0.15288932419196674
cosine = 0.13663075416258424
tfidf = 0.14897159647404337

при К=30
itemitem = 0.14955925563173178
cosine = 0.13917727717923453
tfidf = 0.15093046033300497

при К=40
itemitem = 0.14926542605288756
cosine = 0.14123408423114436
tfidf = 0.14789422135161429

In [80]:
result.apply(lambda x: precision_at_k(x['itemitem'], x['actual'],  5), axis=1).mean()

0.14926542605288756

In [81]:
result.apply(lambda x: precision_at_k(x['cosine'], x['actual'],  5), axis=1).mean()

0.14123408423114436

In [82]:
result.apply(lambda x: precision_at_k(x['tfidf'], x['actual'],  5), axis=1).mean()

0.14789422135161429